In [3]:
import pandas as pd
import requests
import json
from pandas import json_normalize
from pprint import pprint

In [ ]:
gsaf_df = pd.read_csv("Location Counts.csv")

In [ ]:
gsaf_df.head()

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/geocode/json"
lat = []
lng = []
location = []
attack = []
for i in range(len(gsaf_df)):
    try:
        target_city = gsaf_df["index"][i]
        params = {"address": target_city, "key": gkey}
        response = requests.get(base_url, params=params)
        # Convert to JSON
        shark_geo = response.json()
        # Extract lat/lng
        lat.append(shark_geo["results"][0]["geometry"]["location"]["lat"])
        lng.append(shark_geo["results"][0]["geometry"]["location"]["lng"])
        location.append(gsaf_df["index"][i])
        attack.append(gsaf_df["Location"][i])
    except IndexError:
        continue

In [ ]:
print(location[0])

In [ ]:
coordinates = []
for x in range(len(lat)):
    coordinates.append(str(lat[x])+ ", "+ str(lng[x]))
print(coordinates[0])

In [ ]:
# Set up parameters 
target_search = "ice cream shops"
target_radius = 6000
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

count_ice_cream =[]

for x in range(len(location)):
    target_coordinates = coordinates[x]
    params = {
        "location": target_coordinates, 
        "keyword": target_search, 
        "radius": target_radius,
        "key": gkey}
    ice_cream_content = requests.get(base_url, params=params).json()
    nearby_ice_df = json_normalize(ice_cream_content['results'][0:15])
    #Get the number of ice cream shops within 6000ft
    count_ice_cream.append(len(nearby_ice_df))

In [ ]:
ice_cream_df = pd.DataFrame({'location' : location,
                             'Number of Attacks' : attack,
                             'Latitude' : lat,
                             'Longitude' : lng,
                             'Ice Cream Stores in a Mile' : count_ice_cream})

In [ ]:
ice_cream_df.tail()

In [ ]:
ice_cream_df.to_csv("ice_cream.csv", index=False)

In [4]:
# We decided to add this after the other csv was already created. 
# So rather than calling the api again we decided to read in the csv
weather_df = pd.read_csv("ice_cream.csv")

In [5]:
weather_df.head()

,location,Number of Attacks,Latitude,Longitude,Ice Cream Stores in a Mile
0,"New Smyrna Beach, Volusia County",64,29.025819,-80.926998,15
1,"Cocoa Beach, Brevard County",14,28.320007,-80.607551,12
2,"Melbourne Beach, Brevard County",12,28.068350,-80.560330,11
3,"Ponce Inlet, Volusia County",10,29.096373,-80.936998,5
4,"Myrtle Beach, Horry County",8,33.689060,-78.886694,15


In [6]:
api_key = "f50a1b71335ebb06c331a86d1fd9e95f"
units = "imperial"

In [7]:
url = "http://api.openweathermap.org/data/2.5/weather?"

In [8]:
temp = []
for i in range(len(weather_df)):
    lat = weather_df["Latitude"][i]
    lng = weather_df["Longitude"][i]
    query_url = f"{url}lat={lat}&lon={lng}&units={units}&appid={api_key}"
    response = requests.get(query_url).json()
    temp.append(response['main']['temp'])

In [9]:
weather_df["Temperature in Farenheit"] = temp

In [10]:
weather_df.head()

,location,Number of Attacks,Latitude,Longitude,Ice Cream Stores in a Mile,Temperature in Farenheit
0,"New Smyrna Beach, Volusia County",64,29.025819,-80.926998,15,81.45
1,"Cocoa Beach, Brevard County",14,28.320007,-80.607551,12,80.02
2,"Melbourne Beach, Brevard County",12,28.068350,-80.560330,11,80.53
3,"Ponce Inlet, Volusia County",10,29.096373,-80.936998,5,81.61
4,"Myrtle Beach, Horry County",8,33.689060,-78.886694,15,73.92


In [11]:
weather_df.to_csv("weather_df.csv", index=False)